In [1]:
import pandas as pd
import numpy as np

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
reviews = pd.read_csv("Reviews.csv")
df = reviews.copy()
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [6]:
df["ProfileName"] = df["ProfileName"].fillna(df["ProfileName"].mode()[0])
df["Summary"] = df["Summary"].fillna(df["Summary"].mode()[0])

In [7]:
df.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [8]:
df.groupby("Score").count()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text
Score,,,,,,,,,
1,52268,52268,52268,52268,52268,52268,52268,52268,52268
2,29769,29769,29769,29769,29769,29769,29769,29769,29769
3,42640,42640,42640,42640,42640,42640,42640,42640,42640
4,80655,80655,80655,80655,80655,80655,80655,80655,80655
5,363122,363122,363122,363122,363122,363122,363122,363122,363122


In [9]:
df["Score"].replace(1, value="negatif",inplace = True )
df["Score"].replace(2, value="negatif",inplace = True )
df["Score"].replace(4, value="positive",inplace = True )
df["Score"].replace(5, value="positive",inplace = True )

In [10]:
df = df[(df.Score== "negatif")| (df.Score == "positive")]

In [11]:
df.groupby("Score").count()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text
Score,,,,,,,,,
negatif,82037,82037,82037,82037,82037,82037,82037,82037,82037
positive,443777,443777,443777,443777,443777,443777,443777,443777,443777


In [12]:
df_main = pd.DataFrame()
df_main["text"] = df["Text"]
df_main["label"]= df["Score"]

In [13]:
df_main.head()

,text,label
0,I have bought several of the Vitality canned d...,positive
1,Product arrived labeled as Jumbo Salted Peanut...,negatif
2,This is a confection that has been around a fe...,positive
3,If you are looking for the secret ingredient i...,negatif
4,Great taffy at a great price. There was a wid...,positive


In [14]:
#uppercase-lowercase
df_main["text"] = df_main["text"].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [17]:
#deleting punctuations
df_main["text"] = df_main["text"].str.replace("[^\w\s]","")

In [18]:
#deleting numbers
df_main["text"] = df_main["text"].str.replace("\d","")

In [19]:
df_main["text"]

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price there was a wide ...
                                ...                        
568449    great for sesame chickenthis is a good if not ...
568450    im disappointed with the flavor the chocolate ...
568451    these stars are small so you can give  of thos...
568452    these are the best treats for training and rew...
568453    i am very satisfied product is as advertised i...
Name: text, Length: 525814, dtype: object

In [20]:
#STOPWORDS
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords
sw = stopwords.words("english")
df_main["text"] = df_main["text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/halime/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
#deleting less common words
delete = pd.Series(" ".join(df_main["text"]).split()).value_counts()[-1000:]
df_main["text"] = df_main["text"].apply(lambda x: " ".join(x for x in x.split() if x not in delete))

In [22]:
#lemmatization
from textblob import Word
nltk.download("wordnet")
df_main["text"] = df_main["text"].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))

[nltk_data] Downloading package wordnet to /Users/halime/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
#Test-Train Splitting
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
train_x, test_x,train_y,test_y = train_test_split(df_main["text"],df_main["label"])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [24]:
train_y[0:5]

array([0, 1, 1, 1, 1])

In [25]:
test_y[0:5]

array([1, 0, 1, 0, 1])

#COUNT VECTORS

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer()

In [27]:
X_train_count = vectorizer.transform(train_x)
X_test_count = vectorizer.transform(test_x)

#LOGISTIC REGRESSION

In [28]:
from sklearn.linear_model import LogisticRegression
loj = LogisticRegression()
loj_model = loj.fit(X_train_count,train_y)

In [29]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(loj_model, X_test_count,test_y, cv=10).mean()
print(accuracy)

0.9320751096457915


In [30]:
#Sentiment Analysis
texts = df_main["text"]
new_comment = pd.Series(texts)
v = CountVectorizer()
v.fit(train_x)
new_comment = v.transform(new_comment)
sentiment = loj_model.predict(new_comment)
sentiment[0:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [31]:
data = pd.DataFrame({"text": df_main["text"],
                    "sentiment": sentiment})

In [33]:
data.head()

,text,sentiment
0,bought several vitality canned dog food produc...,1
1,product arrived labeled jumbo salted peanutsth...,1
2,confection around century light pillowy citrus...,1
3,looking secret ingredient robitussin believe f...,1
4,great taffy great price wide assortment yummy ...,1


In [39]:
#New Comment
new = pd.Series("That was awful, I do not recommend you to buy this product")
new = v.transform(new)
sentiment = loj_model.predict(new)
sentiment

array([0])

-0 which means negative 